[Документация](https://docs.python.org/3/library/asyncio.html#module-asyncio)

In [13]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

## Срвнение sync/async

In [14]:
sleep_time = 1  # sec

### sync

In [15]:
from time import sleep


def get_some_data():
    print('⌛ Получаем данные...')

    sleep(sleep_time)

    print('✅ Данные получены.')
    return {'status': 'ok', 'data': {'key': 'value'}}

def get_movies(): 
    print('⌛ Получаем список фильмов...')

    sleep(sleep_time)

    print('✅ Список фильмов получен.')
    return ['movie1', 'movie2', 'movie3']

In [16]:
import time

start_time = time.time()
data       = get_some_data()
movies     = get_movies()
end_time   = time.time()
print(f'Время выполнения (sync): {end_time - start_time:.2f} s')

⌛ Получаем данные...
✅ Данные получены.
⌛ Получаем список фильмов...
✅ Список фильмов получен.
Время выполнения (sync): 2.00 s


### async

In [17]:
async def async_get_some_data():
    print('⌛ Получаем данные...')

    await asyncio.sleep(sleep_time)

    print('✅ Данные получены.')
    return {'status': 'ok', 'data': {'key': 'value'}}

async def async_get_movies(): 
    print('⌛ Получаем список фильмов...')

    await asyncio.sleep(sleep_time)

    print('✅ Список фильмов получен.')
    return ['movie1', 'movie2', 'movie3']

In [18]:
import time

start_time = time.time()

# await для двух корутин сразу
data, movies = await asyncio.gather(async_get_some_data(), async_get_movies())
end_time   = time.time()


print(f'Время выполнения (async): {end_time - start_time:.2f} s')

⌛ Получаем данные...
⌛ Получаем список фильмов...
✅ Данные получены.
✅ Список фильмов получен.
Время выполнения (async): 1.00 s


- Async-функция возвращает объект корутины, результат которой нужно ждать (await).
- Корутина выполняется в момент await.
- при await gather() - запускает await сразу для нескольких корутин.
- Пока await из корутины не разрешился, выполнение кода продолжается в цикле событий.
- При завершении корутины, на место await подставляется результат.

## TaskGroup

- Объект задачи содержит внутри себя корутину (`Task(coro=coro, ...)`).
- Задачи внутри TaskGroup запускаются либо при выходе из async with, либо при первом await.

In [19]:
start_time = time.time()

async with asyncio.TaskGroup() as tg: 
    print('Запуск TaskGroup()...')
    
    data_task = tg.create_task(async_get_some_data())
    print('📥 Добавлена задача: get_some_data()')

    movies_task = tg.create_task(async_get_movies())
    print('📥 Добавлена задача: get_movies()')

print('✅ TaskGroup() выполнена')
end_time = time.time()
print(f'Время выполнения (async): {end_time - start_time:.2f} s')
    
print(data_task.result())
print(movies_task.result())


Запуск TaskGroup()...
📥 Добавлена задача: get_some_data()
📥 Добавлена задача: get_movies()
⌛ Получаем данные...
⌛ Получаем список фильмов...
✅ Данные получены.
✅ Список фильмов получен.
✅ TaskGroup() выполнена
Время выполнения (async): 1.00 s
{'status': 'ok', 'data': {'key': 'value'}}
['movie1', 'movie2', 'movie3']
